In [0]:
# Importing Libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import argparse, matplotlib.pyplot as plt



In [0]:
# Network/Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
# Importing Libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import argparse, matplotlib.pyplot as plt



def dataloaders(batch_size, use_cuda):
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)
    
    return train_loader, test_loader


#### path:= "./your_directory/checkpoint_name.tar",
def save_checkpoint(state, checkpoint_path):
    """
    state: checkpoint we want to save
    checkpoint_path: path to save checkpoint
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    return 
 

def load_ckp(checkpoint_path, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_path)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() 
    train_loss = running_loss/len(train_loader)
    return train_loss

#         if batch_idx % log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))

    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss
    
    
def show_losses(train_losses, test_losses):
    fig = plt.figure()
    plt.plot(train_losses, color='blue')
    plt.plot(test_losses, color='red')
    plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
    plt.xlabel('number of training examples seen')
    plt.ylabel('Loss')
    fig.show()
       

In [0]:
batch_size = 64
test_batch_size = 1000
epochs = 14
lr = .1
gamma = 0.7
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

use_cuda = torch.cuda.is_available()
train_loader, test_loader = dataloaders(batch_size, use_cuda)
model = Net().to(device)

# CHANGE AS APPROPRIATE
optimizer = optim.Adadelta(model.parameters(), lr=lr)
# optimizer = optim.LBFGS(model.parameters(), lr=lr, *args)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
# saving checkpoints during/after training;;;can save other variables as well
checkpoint = {'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
train_losses, test_losses = [], []

In [0]:
def main():
    for epoch in range(1, epochs + 1):
            train_loss = train(model, device, train_loader, optimizer, epoch)
            train_losses.append(train_loss)
#             save_checkpoint(checkpoint) #Change file path as appropriate
            test_loss = test(model, device, test_loader)
            test_losses.append(test_loss)
            scheduler.step()
    show_losses(train_losses, test_losses)
        
if __name__ == '__main__':
    main()        


Test set: Average loss: 0.0854, Accuracy: 9732/10000 (97%)


Test set: Average loss: 0.0578, Accuracy: 9810/10000 (98%)


Test set: Average loss: 0.0501, Accuracy: 9836/10000 (98%)


Test set: Average loss: 0.0474, Accuracy: 9847/10000 (98%)


Test set: Average loss: 0.0442, Accuracy: 9851/10000 (99%)


Test set: Average loss: 0.0438, Accuracy: 9855/10000 (99%)


Test set: Average loss: 0.0425, Accuracy: 9853/10000 (99%)


Test set: Average loss: 0.0413, Accuracy: 9856/10000 (99%)

